In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from datetime import datetime

import os
import joblib
import pandas as pd


# Helper Functions
def save_model(model, algorithm: str, experiment: str) -> None:

    today = datetime.today().strftime("%Y-%m-%d")
    
    path = os.path.join(
        os.path.dirname(os.path.dirname(os.getcwd())),
        'models',
        'genre_classification',
        algorithm,
        f'{experiment}_{today}_{algorithm}.pkl')

    print(f"Saving Model:\t{str(path)}")
    joblib.dump(model, path)

In [ ]:
data_path = os.path.join(
    os.path.dirname(os.path.dirname(os.getcwd())),
    'data',
    'processed',
    'track_features_50k.tsv')

df = pd.read_csv(data_path, sep='\t').dropna()

In [ ]:
x = df.drop(
    columns=[
        'popularity', 'duration_ms', 'obtained_date', 
        'genre', 'artist_name', 'track_name', 
        'Unnamed: 0', 'instance_id']
)
y = df[['genre']].values.ravel()

x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y,
    random_state=0)

In [ ]:
x_train.dtypes

acousticness        float64
danceability        float64
energy              float64
instrumentalness    float64
key                  object
liveness            float64
loudness            float64
mode                 object
speechiness         float64
tempo               float64
valence             float64
dtype: object

In [ ]:
categorical_columns = ['mode', 'key']
continuous_columns = [col for col in x_train.columns.to_list() if col not in categorical_columns]

column_transforms = ColumnTransformer([
    ('numeric', StandardScaler(), continuous_columns),
    ('categorical', OneHotEncoder(), categorical_columns)
])

rf_pipe = Pipeline([
    ('transforms', column_transforms),
    ('rf', RandomForestClassifier())
])

mlr_pipe = Pipeline([
    ('transforms', column_transforms),
    ('mlr', LogisticRegression(multi_class='multinomial'))
])

svm_pipe = Pipeline([
    ('transforms', column_transforms), 
    ('svc', SVC())
])

mnb_pipe = Pipeline([
    ('scalar', StandardScaler()),
    ('mnb', MultinomialNB())
])

# vclf_pipe = Pipeline([
#     ('scalar', StandardScaler()),
#     ('vclf', VotingClassifier(
#         estimators=[
#             (
#                 'rf', 
#                 RandomForestClassifier(
#                     bootstrap=False, 
#                     max_depth=70,
#                     max_features='sqrt', 
#                     min_samples_leaf=5,
#                     min_samples_split=5,
#                     n_estimators=300)
#             ),
#             (
#                 'mlr', 
#                 LogisticRegression(
#                     C=10, 
#                     multi_class='multinomial', 
#                     penalty='l1', 
#                     solver='saga')
#             ),
#             (
#                 'svc', 
#                 SVC(
#                     C=1,
#                     kernel='rbf',
#                     gamma='auto')
#             )

#         ],
#     voting='hard',
#     n_jobs=-1))
# ])

In [ ]:
mlr_param_grid = {
    'mlr__solver': ['newton-cg', 'lbfgs', 'saga'],
    'mlr__penalty': ['l1', 'l2'],
    'mlr__C': [0.1, 1, 10],
    'mlr__max_iter': list(range(100, 500, 50))
}
mlr_search = GridSearchCV(
    estimator=mlr_pipe,
    param_grid=mlr_param_grid,
    n_jobs=-1,
    verbose=10)

rf_param_grid = {
    'rf__n_estimators': list(range(100, 500, 50)),
    'rf__max_features': ['auto', 'sqrt'],
    'rf__max_depth': [None] + list(range(10, 110, 20)),
    'rf__min_samples_split': [2,5,10],
    'rf__min_samples_leaf': [1, 2, 5],
    'rf__bootstrap': [False]
}
rf_search = GridSearchCV(
    estimator=rf_pipe,
    param_grid=rf_param_grid,
    n_jobs=-1,
    verbose=10)

svm_param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['rbf'],
    'svc__gamma': ['auto', 'scale']
}
svm_search = GridSearchCV(
    estimator=svm_pipe,
    param_grid=svm_param_grid,
    n_jobs=-1,
    verbose=10)

In [ ]:
mlr_search.fit(x_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[CV 2/5; 2/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l1, mlr__solver=lbfgs
[CV 2/5; 2/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l1, mlr__solver=lbfgs;, score=nan total time=   0.1s
[CV 3/5; 4/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=newton-cg
[CV 3/5; 4/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=newton-cg;, score=0.392 total time=   3.2s
[CV 5/5; 7/144] START mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l1, mlr__solver=newton-cg
[CV 5/5; 7/144] END mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l1, mlr__solver=newton-cg;, score=nan total time=   0.0s
[CV 3/5; 8/144] START mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l1, mlr__solver=lbfgs
[CV 3/5; 8/144] END mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l1, mlr__solver=lbfgs;, score=nan total time=   0.0s
[CV 2/5; 9/144] START mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l1, mlr__solver=saga
[CV 2/5; 9/144] END mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l1, mlr__so

[CV 4/5; 1/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l1, mlr__solver=newton-cg
[CV 4/5; 1/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l1, mlr__solver=newton-cg;, score=nan total time=   0.0s
[CV 1/5; 4/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=newton-cg
[CV 1/5; 4/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=newton-cg;, score=0.386 total time=   3.4s
[CV 5/5; 9/144] START mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l1, mlr__solver=saga
[CV 5/5; 9/144] END mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l1, mlr__solver=saga;, score=0.394 total time=   1.1s
[CV 4/5; 10/144] START mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l2, mlr__solver=newton-cg
[CV 4/5; 10/144] END mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l2, mlr__solver=newton-cg;, score=0.392 total time=   2.7s
[CV 2/5; 12/144] START mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l2, mlr__solver=saga
[CV 2/5; 12/144] END mlr__C=0.1, mlr__max_iter=150, mlr__penal

[CV 4/5; 2/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l1, mlr__solver=lbfgs
[CV 4/5; 2/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l1, mlr__solver=lbfgs;, score=nan total time=   0.1s
[CV 5/5; 4/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=newton-cg
[CV 5/5; 4/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=newton-cg;, score=0.395 total time=   2.9s
[CV 4/5; 6/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=saga
[CV 4/5; 6/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=saga;, score=0.392 total time=   2.8s
[CV 5/5; 11/144] START mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l2, mlr__solver=lbfgs
[CV 5/5; 11/144] END mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l2, mlr__solver=lbfgs;, score=0.395 total time=   1.9s
[CV 5/5; 15/144] START mlr__C=0.1, mlr__max_iter=200, mlr__penalty=l1, mlr__solver=saga
[CV 5/5; 15/144] END mlr__C=0.1, mlr__max_iter=200, mlr__penalty=l1, mlr__solv

[CV 3/5; 2/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l1, mlr__solver=lbfgs
[CV 3/5; 2/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l1, mlr__solver=lbfgs;, score=nan total time=   0.1s
[CV 2/5; 4/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=newton-cg
[CV 2/5; 4/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=newton-cg;, score=0.385 total time=   3.0s
[CV 5/5; 6/144] START mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=saga
[CV 5/5; 6/144] END mlr__C=0.1, mlr__max_iter=100, mlr__penalty=l2, mlr__solver=saga;, score=0.395 total time=   2.6s
[CV 4/5; 11/144] START mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l2, mlr__solver=lbfgs
[CV 4/5; 11/144] END mlr__C=0.1, mlr__max_iter=150, mlr__penalty=l2, mlr__solver=lbfgs;, score=0.391 total time=   1.9s
[CV 2/5; 14/144] START mlr__C=0.1, mlr__max_iter=200, mlr__penalty=l1, mlr__solver=lbfgs
[CV 2/5; 14/144] END mlr__C=0.1, mlr__max_iter=200, mlr__penalty=l1, mlr__sol

In [ ]:
save_model(
    model=mlr_search,
    algorithm='multinomial_logistic_regression',
    experiment='50k_grid_search_results'
)

Saving Model:	{str(path)}


In [ ]:
rf_search.fit(x_train, y_train)

Fitting 5 folds for each of 864 candidates, totalling 4320 fits
[CV 5/5; 75/144] START mlr__C=1, mlr__max_iter=300, mlr__penalty=l1, mlr__solver=saga
[CV 5/5; 75/144] END mlr__C=1, mlr__max_iter=300, mlr__penalty=l1, mlr__solver=saga;, score=0.395 total time=   4.4s
[CV 5/5; 76/144] START mlr__C=1, mlr__max_iter=300, mlr__penalty=l2, mlr__solver=newton-cg
[CV 5/5; 76/144] END mlr__C=1, mlr__max_iter=300, mlr__penalty=l2, mlr__solver=newton-cg;, score=0.395 total time=   3.2s
[CV 4/5; 77/144] START mlr__C=1, mlr__max_iter=300, mlr__penalty=l2, mlr__solver=lbfgs
[CV 4/5; 77/144] END mlr__C=1, mlr__max_iter=300, mlr__penalty=l2, mlr__solver=lbfgs;, score=0.391 total time=   2.2s
[CV 4/5; 81/144] START mlr__C=1, mlr__max_iter=350, mlr__penalty=l1, mlr__solver=saga
[CV 4/5; 81/144] END mlr__C=1, mlr__max_iter=350, mlr__penalty=l1, mlr__solver=saga;, score=0.391 total time=   4.2s
[CV 4/5; 82/144] START mlr__C=1, mlr__max_iter=350, mlr__penalty=l2, mlr__solver=newton-cg
[CV 4/5; 82/144] END 

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ di

[CV 1/5; 113/144] START mlr__C=10, mlr__max_iter=200, mlr__penalty=l2, mlr__solver=lbfgs
[CV 1/5; 113/144] END mlr__C=10, mlr__max_iter=200, mlr__penalty=l2, mlr__solver=lbfgs;, score=0.387 total time=   1.3s
[CV 1/5; 114/144] START mlr__C=10, mlr__max_iter=200, mlr__penalty=l2, mlr__solver=saga
[CV 1/5; 114/144] END mlr__C=10, mlr__max_iter=200, mlr__penalty=l2, mlr__solver=saga;, score=0.387 total time=   8.5s
[CV 1/5; 118/144] START mlr__C=10, mlr__max_iter=250, mlr__penalty=l2, mlr__solver=newton-cg
[CV 1/5; 118/144] END mlr__C=10, mlr__max_iter=250, mlr__penalty=l2, mlr__solver=newton-cg;, score=0.387 total time=   4.2s
[CV 3/5; 119/144] START mlr__C=10, mlr__max_iter=250, mlr__penalty=l2, mlr__solver=lbfgs
[CV 3/5; 119/144] END mlr__C=10, mlr__max_iter=250, mlr__penalty=l2, mlr__solver=lbfgs;, score=0.392 total time=   1.4s
[CV 3/5; 120/144] START mlr__C=10, mlr__max_iter=250, mlr__penalty=l2, mlr__solver=saga
[CV 3/5; 120/144] END mlr__C=10, mlr__max_iter=250, mlr__penalty=l2, m

/usr/local/lib/python3.8/dist-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ di

...

[CV 1/5; 805/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=300;, score=0.428 total time=  25.2s
[CV 2/5; 807/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=400
[CV 2/5; 807/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=400;, score=0.424 total time=  33.8s
[CV 2/5; 809/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=10, rf__n_estimators=100
[CV 2/5; 809/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=10, rf__n_estimators=100;, score=0.422 total time=   7.6s
[CV 2/5; 810/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples

[CV 1/5; 808/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=450
[CV 1/5; 808/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=450;, score=0.427 total time=  38.0s
[CV 1/5; 811/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=10, rf__n_estimators=200
[CV 1/5; 811/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=10, rf__n_estimators=200;, score=0.425 total time=  15.5s
[CV 1/5; 813/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=10, rf__n_estimators=300
[CV 1/5; 813/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=10, rf__n_estimators=300;

[CV 4/5; 835/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=200
[CV 4/5; 835/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=200;, score=0.426 total time=  15.4s
[CV 5/5; 837/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=300
[CV 5/5; 837/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=300;, score=0.428 total time=  22.9s
[CV 5/5; 839/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=400
[CV 5/5; 839/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=40

[CV 2/5; 814/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=10, rf__n_estimators=350;, score=0.426 total time=  26.9s
[CV 2/5; 816/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=10, rf__n_estimators=450
[CV 2/5; 816/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=10, rf__n_estimators=450;, score=0.425 total time=  34.9s
[CV 1/5; 819/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=2, rf__n_estimators=200
[CV 1/5; 819/864] END rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=2, rf__n_estimators=200;, score=0.426 total time=  16.3s
[CV 1/5; 821/864] START rf__bootstrap=False, rf__max_depth=90, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_sample

GridSearchCV(estimator=Pipeline(steps=[('transforms',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         StandardScaler(),
                                                                         ['acousticness',
                                                                          'danceability',
                                                                          'energy',
                                                                          'instrumentalness',
                                                                          'liveness',
                                                                          'loudness',
                                                                          'speechiness',
                                                                          'tempo',
                                                                     

In [ ]:
save_model(
    model=rf_search,
    algorithm='random_forest',
    experiment='grid_search_results'
)

Saving Model:	/notebooks/capstone/models/genre_classification/random_forest/grid_search_results_2022-01-23_random_forest.pkl


In [ ]:
svm_search.fit(x_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=Pipeline(steps=[('transforms',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         StandardScaler(),
                                                                         ['acousticness',
                                                                          'danceability',
                                                                          'energy',
                                                                          'instrumentalness',
                                                                          'liveness',
                                                                          'loudness',
                                                                          'speechiness',
                                                                          'tempo',
                                                                     

In [ ]:
save_model(
    model=svm_search,
    algorithm='support_vector_machine',
    experiment='grid_search_results'
)

Saving Model:	/notebooks/capstone/models/genre_classification/support_vector_machine/grid_search_results_2022-01-23_support_vector_machine.pkl
